# Pytorch

It is easy to export a Pytorch model to ONNX because it is built into the API. The Pytorch documentation provides a good example on how to perform this conversion.

This is a simplified example:

In [1]:
import numpy as np
import torch
import torchvision

In [2]:
dummy_input = torch.randn(1,3,224,224, device='cpu')
model = torchvision.models.alexnet(pretrained=True)

In [3]:
input_names = ['input_alexnet'] + ['layer_%d' % i for i in range(16)]
output_name = ['output_alexnet'] 

In [4]:
torch.onnx.export(model, dummy_input, 'alexnet_onnx.onnx', verbose=True, input_names= input_names, output_names=output_name)

graph(%input_alexnet : Float(1, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=0, device=cpu),
      %layer_0 : Float(64, 3, 11, 11, strides=[363, 121, 11, 1], requires_grad=1, device=cpu),
      %layer_1 : Float(64, strides=[1], requires_grad=1, device=cpu),
      %layer_2 : Float(192, 64, 5, 5, strides=[1600, 25, 5, 1], requires_grad=1, device=cpu),
      %layer_3 : Float(192, strides=[1], requires_grad=1, device=cpu),
      %layer_4 : Float(384, 192, 3, 3, strides=[1728, 9, 3, 1], requires_grad=1, device=cpu),
      %layer_5 : Float(384, strides=[1], requires_grad=1, device=cpu),
      %layer_6 : Float(256, 384, 3, 3, strides=[3456, 9, 3, 1], requires_grad=1, device=cpu),
      %layer_7 : Float(256, strides=[1], requires_grad=1, device=cpu),
      %layer_8 : Float(256, 256, 3, 3, strides=[2304, 9, 3, 1], requires_grad=1, device=cpu),
      %layer_9 : Float(256, strides=[1], requires_grad=1, device=cpu),
      %layer_10 : Float(4096, 9216, strides=[9216, 1], requires_gra

In [14]:
import torch.nn as nn
import torch.nn.functional as F

class LSTM(nn.Module):
    def __init__(self, in_ts, in_ch, out_dim, p=0.0):
        super(LSTM, self).__init__()
        self.lstm1 = nn.LSTM(in_ts,  128, 1, dropout=0.3)
        self.lstm2 = nn.LSTM(128, 256, 1, dropout=0.4)
        self.fc1   = nn.Linear(in_ch*256, out_dim)
        self.drop1 = nn.Dropout(p=p//2)

    def forward(self,x):
        x = F.tanh(self.lstm1(x)[0])
        x = F.tanh(self.lstm2(x)[0])
        x = torch.flatten(x, 1)
        x = self.drop1(x)
        x = self.fc1(x)
        return x

MODEL_LSTM = LSTM(100,1,100)
print(MODEL_LSTM)

LSTM(
  (lstm1): LSTM(100, 128, dropout=0.3)
  (lstm2): LSTM(128, 256, dropout=0.4)
  (fc1): Linear(in_features=256, out_features=100, bias=True)
  (drop1): Dropout(p=0.0, inplace=False)
)


In [15]:
dummy_input = torch.randn(1,1,100, device='cpu')
torch.onnx.export(MODEL_LSTM, dummy_input, 'lstm_onnx.onnx')

C:\Users\lospa\anaconda3\envs\dl\lib\site-packages\torch\nn\functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
C:\Users\lospa\anaconda3\envs\dl\lib\site-packages\torch\onnx\symbolic_opset9.py:1932: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn("Exporting a model to ONNX with a batch_size other than 1, " +


# TensorFlow

Exporting a TensorFlow neural network to ONNX takes a bit longer than with Pytorch, but it is still straightforward. 

Tensorflow and ONNX both define their own graph format to represent to model. You can use tensorflow-onnx to export a Tensorflow model to ONNX.

> Procedures to convert tensorflow model

>   - get tensorflow model
>    - convert to ONNX
>    - validate

Tensorflow uses several file formats to represent a model, such as checkpoint files, graph with weight(called frozen graph next) and saved_model, and it has APIs to generate these files. And tensorflow-onnx can accept all the three formats to represent a Tensorflow model, the format "saved_model" should be the preference since it doesn't require the user to specify input and output names of graph.

*Resources:*
https://github.com/onnx/tutorials

In [5]:
import os
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import onnxruntime

x = np.random.randn(1,224,224,3).astype(np.float32)
x = preprocess_input(x)
print(x.dtype)

float32


In [6]:
model = ResNet50(weights='imagenet')
preds = model.predict(x)
print('Keras Predicted:', decode_predictions(preds, top=3)[0])
model.save('resnet_model')

Keras Predicted: [('n04404412', 'television', 0.07654696), ('n03782006', 'monitor', 0.07245921), ('n04286575', 'spotlight', 0.053543683)]
INFO:tensorflow:Assets written to: resnet_model\assets


In [8]:
import tf2onnx
import onnxruntime as rt

spec = (tf.TensorSpec((None, 224, 224, 3), tf.float32, name="input"),)
output_path = "resnet_model.onnx"

model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=output_path)
output_names = [n.name for n in model_proto.graph.output]

In [9]:
providers = ['CPUExecutionProvider']
m = rt.InferenceSession(output_path, providers=providers)
onnx_pred = m.run(output_names, {"input": x})

print('ONNX Predicted:', decode_predictions(onnx_pred[0], top=3)[0])

# make sure ONNX and keras have the same results
np.testing.assert_allclose(preds, onnx_pred[0], rtol=1e-5)

ONNX Predicted: [('n04404412', 'television', 0.07654693), ('n03782006', 'monitor', 0.07245866), ('n04286575', 'spotlight', 0.053543683)]


AssertionError: 
Not equal to tolerance rtol=1e-05, atol=0

Mismatched elements: 7 / 1000 (0.7%)
Max absolute difference: 5.5134296e-07
Max relative difference: 1.302095e-05
 x: array([[3.322915e-05, 4.303578e-04, 1.908984e-04, 1.436556e-04,
        2.868907e-04, 5.629365e-05, 1.296373e-04, 2.399527e-04,
        5.998426e-05, 1.996869e-04, 1.583722e-04, 2.685153e-04,...
 y: array([[3.322940e-05, 4.303595e-04, 1.909006e-04, 1.436563e-04,
        2.868932e-04, 5.629393e-05, 1.296389e-04, 2.399538e-04,
        5.998453e-05, 1.996882e-04, 1.583732e-04, 2.685163e-04,...

In [ ]:
# python -m tf2onnx.convert --opset 13 --saved-model {"resnet_model"} --output  {"resnet_model.onnx"}